# BEFORE YOU BEGIN

Run `Cell -> Run All` from the notebook menu bar. This will load needed extensions and set up the database that the document will use.

_Your queries will not work if you do not do this!_

Note that `Run All` will probably jump you to the end of the document and you will have to scroll back here when it completes.

# Overview

This notebook is intended to provide a _non-comprehensive_ introduction to SQL and relational database concepts. 

Specifically, it was created to show a person with some technical background but no familiarity with SQL everything needed to construct queries to address use cases like:

- _Given a table of event logs associated with users, I need to see events triggered by a specific set of users_
- _Given a table of event logs associated with users, I need the set of users assocaited with a specific event recorded during a given period of time_

To address these use cases, the document will cover:

1. Basic relational database structure
1. `SELECT`ing columns from a table
1. `ORDER`ing and `LIMIT`ing column data
1. `GROUP`ing and aggregating column data
1. Filtering results to show only columns `WHERE` the value matches one or more criteria, including being `IN` a given list
1. `JOIN`ing tables together


_words formatted like `THIS` are SQL statements you'll see later_

## Using this document

This document is a Jupyter Notebook, which means it is an interactive mix of markdown (like this cell), Python, and SQL. 

The great thing about notebooks like this is that you can make changes and mess around with it. You can click any cell to make changes to it, then run `Ctrl+Enter` to execute it and see the output. 

**Pro-tip:** want to experiment with a query without losing the original? You can `Edit->Copy`/`Edit->Paste` any cell to duplicate it!

The notebook uses an in-memory database created just for it, so experiment away! If you want to start over, just re-run the big setup cell in the next section.

---

# I. Creating and populating tables (optional)

_**Don't panic!** This section is long, but skipable!_

The cells in this section set up the database and populate it with users and events. Knowing how to do that isn't needed to address the use cases listed above, so you can either skip to the next section or read on for some bonus knowledge!

Before we can query any data, we need... data to query! 

We're going to use a simple database with two tables, one for users and one for events. Each event will be associated with a user and a date/time. Why two tables? More on that later... 

_Note: the cells below contain a mix of Python and SQL code. The lines that begin with `%` or `%%` are instructions specific to notebooks, not normally found in either language. The `%%capture` you'll see at the top of some cells just instructs the notebook to not show output for that cell._

In [2]:
%%capture 
# The %%capture above just tells the notebook to not print output for 
# this cell, since it's just a bunch of generic "operation complete"
# messages.


# These load some extra functionality we'll need, like 
# an extension that causes lines prefixed with %sql
# (or entire cells when they begin with %%sql) to be
# sent to the database instead of Python
%load_ext sql
%sql sqlite://
from random import choice, randint

# Note that the code in this cell is Python, not SQL. 
# The code in the next cell will be interpreted as
# SQL (as denoted by the %%sql at the top)

# "drop" means "remove" in SQL. In this case, we're removing
# the tables if they already exist. This is so you can easily
# re-create your tables by re-running this cell, in case you
# want to experiment with adding columns, for example. Be sure
# to run the next cell too, so your tables have data in them!
#
# Hopefully this is obvious, but...
#
#  *** BE VERY CAREFUL WITH DROP STATEMENTS IN REAL LIFE ***
#

%sql DROP TABLE IF EXISTS users;
%sql DROP TABLE IF EXISTS events;


# Now we can (re-)create our tables. 
# The lines inside the parentheses are column definitions.
# These consist of the column name followed by one or more 
# properties for it. The only required property is the "type"
# of data the column will store, i.e. integer, text, etc. 
#
# The specific data types supported will vary depending on which
# vendor's SQL database you are using. This notebook uses SQLite,
# and you can learn more about how it uses data types here:
# https://sqlite.org/datatype3.html
#
# Note that the first column in each table is a number with the
# "PRIMARY KEY" property. You'll learn more about this, and the 
# FOREIGN KEY in the events table, soon.
#
# Quick syntax note: the `\` at the end of each line just tells
# the notebook to keep treating the text as SQL instead of going
# back to Python.

%sql CREATE TABLE users (                 \
    id INTEGER PRIMARY KEY,               \
    name TEXT                             \
);

%sql CREATE TABLE events (                \
    id INTEGER PRIMARY KEY,               \
    event_type TEXT,                      \
    timestamp TEXT,                       \
    user INTEGER,                         \
    FOREIGN KEY(user) REFERENCES user(id) \
);


# Now we'll use Python to quickly make up some data to put
# in our new tables!

# First we'll make up some names of users and events to log, 
# and a set of dates to distribute the logs across.
#
# Feel free to change these to suit your taste! Just make sure
# you have significantly more users than events, so that some
# events are logged multiple times with different users!
USER_NAMES = [
    'sally',
    'bob',
    'jane',
    'norma',
    'mercutio',
    'andre',
    'shayna',
    'asuka',
    'bianca',
]

EVENT_NAMES = [
    "defragged the widget",
    "gleamed the cube",
    "hooped the frood",
    "walked the line",
]

EVENT_DATES = [
    '2020-01-01 00:00:00',
    '2020-01-02 00:00:00',
    '2020-01-03 00:00:00',
    '2020-01-04 00:00:00',
]

# To ensure that all users are represented in the
# events log, we'll create this many events for each.
EVENTS_PER_USER = 2

# Now we're ready to start populating our tables!  

# This will create a row in the users table for each user name
for name in USER_NAMES:
    # Adding a row to a table is done with an INSERT instruction. 
    # Note that the values in the VALUES list correspond to the 
    # columns specified in our CREATE statement. 
    # The first value, corresponding to the id/primary key column,
    # is `null` because this causes the database to assign its
    # value automatically.
    %sql INSERT INTO users VALUES(null, :name);

# Finally, we'll randomly create some events
for user_index in range(len(USER_NAMES)):
    # In the USER_NAMES list, Python has assigned each
    # name a sequential index number starting at 0.
    # The database does the same thing with user ID
    # (primary key) values, but starting at 1 instead.
    # In other words, we can get a user's ID in the 
    # database by adding 1 to its index value. 
    user_id = user_index + 1
    
    # This block of code will be repeated 
    # EVENTS_PER_USER times.
    for i in range(EVENTS_PER_USER):
        # choice(...) picks a random item from the given list
        event_name = choice(EVENT_NAMES)
        event_date = choice(EVENT_DATES)
        %sql INSERT INTO events VALUES(null, :event_name, date(:event_date), :user_id);

# II. Database normalization: why do we event have two tables?

Before we go any further, let's talk about the basics of relational database design. 

The code in the previous cell created two tables, a `users` table that looks like this:

<table>
    <tr><th>id</th><th>name</th></tr>
    <tr><td>1</td><td>sally</td></tr>
    <tr><td>2</td><td>bob</td></tr>
    <tr><td>3</td><td>jane</td></tr>
    <tr><td colspan=2>...</td></tr>
</table>

...and an `events` table that looks (something) like this:

<table>
    <tr><th>id</th><th>event_type</th><th>timestamp</th><th>user</th></tr>
    <tr><td>1</td><td>hooped the frood</td><td>2020-01-04 00:00:00</td><td>2</td></tr>
    <tr><td>2</td><td>walked the line</td><td>2020-01-02 00:00:00</td><td>3</td></tr>
    <tr><td>3</td><td>gleamed the cube</td><td>2020-01-03 00:00:00</td><td>1</td></tr>
    <tr><td>4</td><td>hooped the frood</td><td>2020-01-01 00:00:00</td><td>3</td></tr>
    <tr><td colspan=4>...</td></tr>
</table>

## Primary keys

Note that each user and event has an `id` that uniquely identifies it. These unique identifiers are called _primary keys_. The primary key doesn't have to be called `id`, but it usually is.

## Foreign keys

The `event` table also has a column called `user`. The values in this column are primary key values from the `users` table. So the first event is associated with the user with primary key `2`, aka bob, the second event is associated with user ID 3 aka jane, and so on.
  
A value that references the primary key from another table like this is called a _foreign key_.
  
The foreign key _relates_ the `events` table to the `users` table.

## Ok... but why?

So why not just have it all the information in our `events` table, like this?

<table>
    <tr><th>id</th><th>event_type</th><th>timestamp</th><th>user</th></tr>
    <tr><td>1</td><td>hooped the frood</td><td>2020-01-04 00:00:00</td><td>bob</td></tr>
    <tr><td>2</td><td>walked the line</td><td>2020-01-02 00:00:00</td><td>jane</td></tr>
    <tr><td>3</td><td>gleamed the cube</td><td>2020-01-03 00:00:00</td><td>sally</td></tr>
    <tr><td>4</td><td>hooped the frood</td><td>2020-01-01 00:00:00</td><td>jane</td></tr>
    <tr><td colspan=2>...</td></tr>
</table>

Wouldn't that be simpler?

In this trivial example... you actually could make the case that it would, but normally user records are more complicated than a single name. Let's talk about how the relational approach helps manage data, espeically in real-world use cases.

### Reason 1:  Specificity

What if instead of just a name, we stored a first name, last name, title, phone number, etc for each user?

Now suddenly our single-table layout looks like this:

<table>
    <tr><th>id</th><th>event_type</th><th>timestamp</th><th>first</th><th>last</th><th>title</th><th>phone</th></tr>
    <tr><td>1</td><td>hooped the frood</td><td>2020-01-04 00:00:00</td><td>bob</td><td>jones</td><td>project manager</td><td>xxx-xxx-xxxx</td></tr>
    <tr><td>2</td><td>walked the line</td><td>2020-01-02 00:00:00</td><td>jane</td><td>janeson</td><td>data analyst</td><td>xxx-xxx-xxxx</td></tr>
    <tr><td>3</td><td>gleamed the cube</td><td>2020-01-03 00:00:00</td><td>sally</td><td>ngoma</td><td>cube gleamer</td><td>xxx-xxx-xxxx</td></tr>
    <tr><td>4</td><td>hooped the frood</td><td>2020-01-01 00:00:00</td><td>jane</td><td>janeson</td><td>data analyst</td><td>xxx-xxx-xxxx</td></tr>
    <tr><td colspan=4>...</td></tr>
</table>

For starters, it hardly even makes sense to call this table `events` any more, because it has more fields devoted to describing users than events. 

So one simple advantage of using a relational model is that it reduces the amount of data being crammed into a given table by keeping each table focused on attributes pertaining to a specific thing. 

That's useful, but there's something even more central to what makes the relational model practical.

### Reason 2: Single source of truth 

What if you wanted to change Bob Jone's phone number in this table? You could do that by just updating the `phone` field in row `1`. Clumsy, but do-able.

Now what if you wanted to change Jane Johnson's phone number? Note that she appears in two rows of the table. Better be sure to update them both! 

What if you just wanted to look up Jane's title? Which record in the `events` table is authoritative for her?

The relational design we're using solves both of these problems: each table is limited to properties of a specific  class of thing (users or events), and the properties of each thing are described exactly once.

## Optional further reading

Designing relational databases with just the right amount of abstraction (there's such a thing as too much!) is called _normalization_ and [plenty has been written about it online](https://duckduckgo.com/?q=relational+database+normalization&t=hk&ia=web). 

A final thought exercise: in the example above, in additon to moving user data to its own table, we _could_ do the same for titles ("project manager", "data analyst", etc). After all, users relatedto titles similarly to the way events relate to users. But in this case it wouldn't gain us much. Can you say why? Can you imagine a scenario where it _would_ be helpful to do this? _(hint: it has to do with how many properties of a title we care about storing)_

---

# III. `SELECT`ing rows from a table

Enough theory, let's see what we've done!

In SQL, `SELECT` means "show me", so

```sql
SELECT * FROM events;
```

means "show me all columns (`*`) from `events`.

You can also request specific columns, like this:

```sql
SELECT event_type, timestamp FROM events;
```

Let's see that first query in action:

In [21]:
%%sql 

SELECT * FROM events;

 * sqlite://
Done.


id,event_type,timestamp,user
1,defragged the widget,2020-01-03,1
2,hooped the frood,2020-01-02,1
3,gleamed the cube,2020-01-02,2
4,walked the line,2020-01-04,2
5,walked the line,2020-01-01,3
6,gleamed the cube,2020-01-02,3
7,hooped the frood,2020-01-04,4
8,walked the line,2020-01-02,4
9,walked the line,2020-01-01,5
10,defragged the widget,2020-01-03,5


## Try it!

Edit the cell below and add your `SELECT` statement that gets all columns from the `users` table, then modify it to only take the `name` column. 

In [ ]:
%%sql

/* "Show me all columns for every row in the users table"   */
/* Write your query below, then press Ctrl+Enter to run it! */



<div class="answer">SELECT * FROM users;
SELECT name FROM users;</div>

## Refining `SELECT` with `ORDER BY` and `LIMIT`

Events in our event log appear in the order in which they were inserted, and since each event's date was randomly generated, chances are that means they are not chronological. 

We can fix that by adding `ORDER BY` to the query:

```sql
SELECT event_name, timestamp FROM events ORDER BY timestamp
```

Let's see that query in action:

In [5]:
%%sql

/* 
# Syntax note: putting FROM and ORDER BY on their 
# own lines is optional, but as our queries get longer, 
# the multi-line format will make them easier to read.
*/
SELECT event_type, timestamp 
    FROM events 
    ORDER BY timestamp

 * sqlite://
Done.


event_type,timestamp
walked the line,2020-01-01
walked the line,2020-01-01
defragged the widget,2020-01-01
gleamed the cube,2020-01-01
hooped the frood,2020-01-01
hooped the frood,2020-01-02
gleamed the cube,2020-01-02
gleamed the cube,2020-01-02
walked the line,2020-01-02
defragged the widget,2020-01-03


Note that results are in ascending order by default. You can request descending order by adding the keyword `desc`:

```sql
SELECT event_type, timestamp FROM events ORDER BY timestamp
```

This combines nicely with the `LIMIT` statement, which tells the database to limit results to a maximum number of records. 

So, suppose you want to see just the 5 most recent events...

In [6]:
%%sql

SELECT event_type, timestamp 
    FROM events 
    ORDER BY timestamp desc
    LIMIT 5

 * sqlite://
Done.


event_type,timestamp
walked the line,2020-01-04
hooped the frood,2020-01-04
gleamed the cube,2020-01-04
defragged the widget,2020-01-03
defragged the widget,2020-01-03


---

# IV. Limiting results with `WHERE`

What if we want to see all events of a specific type?

`WHERE` lets you add criteria to your `SELECT` statement using comparisons like `=`, `!=`, `<`, `>`, `<=`, etc.

So to get the timestamp for each "defragged the widget" event, you would run

```sql
SELECT timestamp FROM events 
    WHERE event_type = 'defragged the widget';
```

You can combine `WHERE` criteria with boolean operators (`AND`, `OR`, `NOT`)

```sql
SELECT id FROM events 
    WHERE event_type = 'defragged the widget'
          AND NOT timestamp = '2020-01-04 00:00:00';
```

## Limiting to sets of values with `IN`

The query above gets the `id` of each "defragged the widget" event except those with a specific date. 

What if we wanted to exclude two dates instead of one?

You could do that with more booleans:

```sql
SELECT id FROM events 
    WHERE event_type = 'defragged the widget'
          AND NOT (timestamp = '2020-01-04 00:00:00' OR timestamp = '2020-01-03 00:00:00') ;
```

...but SQL give you a more elegant way with `IN`:

```sql
SELECT id FROM events 
    WHERE event_type = 'defragged the widget'
          AND timestamp NOT IN ('2020-01-04 00:00:00', '2020-01-03 00:00:00') ;
```

## Try it! 

Combine what you've learned so far to write a query that says "show me the timestamp of the most recent event with type 'hooped the frood' _or_ 'gleamed the cube'".

In [ ]:
%%sql

/* "Show me the timestamp of the most recent event with type 'hooped the frood' or 'gleamed the cube'"   */
/* Write your query below, then press Ctrl+Enter to run it! */


<div class="answer">SELECT timestamp FROM events
  GROUP BY event_type
  WHERE event_type IN ('hooped the frood', 'gleamed the cube')
  ORDER BY timestamp desc
  LIMIT 1
</div>

---
# V. `GROUP`ing and aggregating

Suppose you wanted to know _how many_ events of each type have occurred? 

Getting the answer to that boils down to two operations:
1. _Group_ the events by type
2. _Count_ the `id`s in each group

As a SQL query, this would look like:

```sql
SELECT count(id) FROM events
  GROUP BY event_type;
```

note that the counting is done by a `count()` function. It's important to note that while `GROUP` is a standard part of SQL, the specific functions available to you for aggregation can depend on the database you're using. The aggregation functions available in SQLite are listed [here](https://www.sqlite.org/lang_aggfunc.html)

Grouping an aggregation open up a lot of options. See the next cell for another example to spark your imagination.

In [8]:
%%sql

SELECT timestamp, group_concat(event_type) 
    FROM events 
    GROUP BY timestamp;

 * sqlite://
Done.


timestamp,group_concat(event_type)
2020-01-01,"walked the line,walked the line,defragged the widget,gleamed the cube,hooped the frood"
2020-01-02,"hooped the frood,gleamed the cube,gleamed the cube,walked the line"
2020-01-03,"defragged the widget,defragged the widget,hooped the frood,gleamed the cube,defragged the widget,defragged the widget"
2020-01-04,"walked the line,hooped the frood,gleamed the cube"


---

# VI. Renaming columns in `SELECT` output with `as`

The query in the previous cell returns a comma-separated list of event types for each day. But note the name of that second column, `group_concat(event_type)`. That's... descriptive, but maybe not as helpful as it could be. 

Fortunately it's easy to fix! Any column in the output of a `SELECT` statement can be given a name, like this..

In [30]:
%%sql

SELECT timestamp as 'date', group_concat(event_type) as 'logged events' 
    FROM events 
    GROUP BY timestamp;

 * sqlite://
Done.


date,logged events
2020-01-01,"walked the line,walked the line,defragged the widget,gleamed the cube,hooped the frood"
2020-01-02,"hooped the frood,gleamed the cube,gleamed the cube,walked the line"
2020-01-03,"defragged the widget,defragged the widget,hooped the frood,gleamed the cube,defragged the widget,defragged the widget"
2020-01-04,"walked the line,hooped the frood,gleamed the cube"


## Try it!

Use the cell below to create a query that says "Show me each event type, and the number of logs of that type in a column labeled 'instances', sorted by instance count from highest to lowest"

In [ ]:
%%sql

/* "Show me each event type, and the number of logs of that type in a column labeled 'instances', sorted by instance count from highest to lowest"   */
/* Write your query below, then press Ctrl+Enter to run it! */


<div class="answer">SELECT event_type, count(id) as instances 
    FROM events
    GROUP BY event_type
    ORDER BY instances desc;</div>

### Bonus challenge

Try writing another query that says "Show me each event type, and the number of _distinct_ dates on which that event was seen". _hint: this will require a special keyword we haven't covered yet, but you'll see it mentioned early in the_ [SQLite aggregation functions doc](https://www.sqlite.org/lang_aggfunc.html)

In [ ]:
%%sql

/* "Show me each event type, and the number of distinct dates on which that event was seen"   */
/* Write your query below, then press Ctrl+Enter to run it! */


<div class="answer">SELECT event_type, count(distinct timestamp) as instances 
    FROM events
    GROUP BY event_type
    ORDER BY instances desc;</div>

---

# VII. Pulling things together with `JOIN... ON` statements

So far we've worked exclusively with one table for our examples. But of course the real power of a relational database comes from being able to connect, in our case, events with users. You can do this with a `JOIN... ON`:

```sql
SELECT users.name, events.event_type, events.timestamp 
    FROM users JOIN events 
        ON users.id = events.user
```

The `ON` portion here is key, because it tells the database _how_ the two tables are related. It's essentially creating a table like this', kind of like the giant table from the section on normalization, but only created when we need it:

<table>
    <tr><th>user.id</th><th>user.name</th><th>events.id</th><th>events.event_type</th><th>events.timestamp</th><th>events.user</th></tr>
    <tr><td style='color:green'>2</td><td>bob</td><td>1</td><td>hooped the frood</td><td>2020-01-04 00:00:00</td><td style='color:green'>2</td></tr>
    <tr><td style='color:red'>3</td><td>jane</td><td>2</td><td>walked the line</td><td>2020-01-02 00:00:00</td><td style='color:red'>3</td></tr>
    <tr><td style='color:orange'>1</td><td>sally</td><td>3</td><td>gleamed the cube</td><td>2020-01-03 00:00:00</td><td style='color:orange'>1</td></tr>
    <tr><td style='color:red'>3</td><td>jane</td><td>4</td><td>hooped the frood</td><td>2020-01-01 00:00:00</td><td style='color:red'>3</td></tr>
    <tr><td colspan=4>...</td></tr>
</table>

Everything you've learned so far can be combined with `JOIN... ON`!

In [11]:
%%sql

/* 
"Show me the user responsible for the most 'walked the line' and/or 'hooped the frood' events"
*/
SELECT users.name, count(events.id) as num_events
    FROM users JOIN events
        ON users.id = events.user
    WHERE events.event_type IN ('walked the line', 'hooped the frood')
    ORDER BY num_events
    LIMIT 1


 * sqlite://
Done.


name,num_events
sally,8


## `INNER` vs `OUTER JOIN`s

Technically the join shown above is called an _inner join_ (you could actually write `INNER JOIN` instead of just `JOIN` if you wanted to be explicit).

This is what you will want most of the time (hence it being the default), but are other joins, the most common being the _left outer join_. 

What's the difference? Well, with the data we currently have, probably nothing. Note that the following queries return the same result:

In [12]:
%%sql

SELECT users.name, events.event_type, events.timestamp 
    FROM users INNER JOIN events 
        ON users.id = events.user
    ORDER BY users.id desc
    LIMIT 5;

 * sqlite://
Done.


name,event_type,timestamp
bianca,defragged the widget,2020-01-03
bianca,gleamed the cube,2020-01-04
asuka,gleamed the cube,2020-01-01
asuka,hooped the frood,2020-01-01
shayna,gleamed the cube,2020-01-03


In [13]:
%%sql

/* # note the different JOIN type */
SELECT users.name, events.event_type, events.timestamp 
    FROM users LEFT OUTER JOIN events 
        ON users.id = events.user
    ORDER BY users.id desc
    LIMIT 5;

 * sqlite://
Done.


name,event_type,timestamp
bianca,defragged the widget,2020-01-03
bianca,gleamed the cube,2020-01-04
asuka,gleamed the cube,2020-01-01
asuka,hooped the frood,2020-01-01
shayna,defragged the widget,2020-01-03


...but suppose we add a new user who isn't associated with any events?

In [14]:
%%sql 

INSERT INTO users VALUES(null, 'neo');
SELECT * 
    FROM users
    ORDER BY id desc
    LIMIT 1;

 * sqlite://
1 rows affected.
Done.


id,name
10,neo


Our inner join looks the same as before

In [15]:
%%sql

SELECT users.name, events.event_type, events.timestamp 
    FROM users INNER JOIN events 
        ON users.id = events.user
    ORDER BY users.id desc
    LIMIT 5;

 * sqlite://
Done.


name,event_type,timestamp
bianca,defragged the widget,2020-01-03
bianca,gleamed the cube,2020-01-04
asuka,gleamed the cube,2020-01-01
asuka,hooped the frood,2020-01-01
shayna,gleamed the cube,2020-01-03


...while the left outer join includes the new user

In [16]:
%%sql

SELECT users.name, events.event_type, events.timestamp 
    FROM users LEFT OUTER JOIN events 
        ON users.id = events.user
    ORDER BY users.id desc
    LIMIT 5;

 * sqlite://
Done.


name,event_type,timestamp
neo,None,None
bianca,defragged the widget,2020-01-03
bianca,gleamed the cube,2020-01-04
asuka,gleamed the cube,2020-01-01
asuka,hooped the frood,2020-01-01


## To sumarize, the difference between and _inner_ and _outer_ join...

...lies in how rows that _don't_ match the `ON` criteria are treated. An _inner_ join ignores such records, while a _left outer_ join includes them.

Neo's user ID is not referenced in `events.user`, so he is ignored by the outer join, but included in the left outer join. 

Again, the former behavior is usually what you want, but if you needed to, say, list all users and the number of events associated with them _even if that number is zero_, a _left outer_ join like this would save your day:

In [17]:
%%sql

/* # Try removing `LEFT OUTER` and see how the results change */
SELECT users.name, count(events.id) as num_events
    FROM users LEFT OUTER JOIN events
        ON users.id = events.user
    GROUP BY events.user
    ORDER BY num_events
    LIMIT 3
    

 * sqlite://
Done.


name,num_events
neo,0
sally,2
bob,2


## Optional further reading

You can see more examples of inner joins, left outer joins, and more [here](https://www.w3schools.com/sql/sql_join.asp)

---

# VIII. Applying what you've learned

Let's look back at those original use cases:

- _Given a table of event logs associated with users, I need to see events triggered by a specific set of users_
- _Given a table of event logs associated with users, I need the set of users assocaited with a specific event recorded during a given period of time_


## Try it yourself!
You now have all the tools you need to solve the first problem. Try it in the cell below. As a hint, key components of your query will include: `JOIN`, and `IN`.

In [ ]:
%%sql

/* "Show me events triggered by Sally and Bob only" */
/* Write your query below, then press Ctrl+Enter to run it! */



<div class="answer">SELECT events.timestamp, events.event_type, users.name
    FROM events JOIN users
        ON events.user = users.id
    WHERE users.name IN ('sally', 'bob');</div>

The second use case is trickier because it requires checking whether an event occurred within a given date range. 

The exact way you handle date comparisons like this will vary depending on the database you are using. In SQLite you can make things a bit easier by converting dates to numbers (seconds since midnight UTC on 1st Jan 1970, aka ["unix time"](https://sqlite.org/lang_datefunc.html)), which you can then compare using `>`, `<`, etc. This conversion is done with the `strftime` function, documented along with other date/time-related functions [here](https://sqlite.org/lang_datefunc.html).

For example, here's how you could use it to select all event records on or after `2020-01-03 00:00:00`:

In [19]:
%%sql

SELECT * from events
    WHERE strftime('%s', events.timestamp) >= strftime('%s', '2020-01-03 00:00:00')
    ORDER BY events.timestamp;   

 * sqlite://
Done.


id,event_type,timestamp,user
1,defragged the widget,2020-01-03,1
10,defragged the widget,2020-01-03,5
12,hooped the frood,2020-01-03,6
13,gleamed the cube,2020-01-03,7
14,defragged the widget,2020-01-03,7
17,defragged the widget,2020-01-03,9
4,walked the line,2020-01-04,2
7,hooped the frood,2020-01-04,4
18,gleamed the cube,2020-01-04,9


Given this knowledge, now have a go at writing a query to address the second use case.

In [56]:
%%sql

/* "Show me users and timestamps for `gleamed the cube` events between January 1st and 3rd (inclusive) */
SELECT users.name, events.timestamp FROM 
    users JOIN events ON users.id = events.user
    WHERE strftime('%s', events.timestamp) >= strftime('%s', '2020-01-01 00:00:00')
        AND strftime('%s', events.timestamp) <= strftime('%s', '2020-01-03 00:00:00')
        AND event_type = 'gleamed the cube'

 * sqlite://
Done.


name,timestamp
bob,2020-01-02
jane,2020-01-02
shayna,2020-01-03
asuka,2020-01-01


<div class="answer">SELECT users.name, events.timestamp FROM 
    users JOIN events ON users.id = events.user
    WHERE strftime('%s', events.timestamp) >= strftime('%s', '2020-01-01 00:00:00')
        AND strftime('%s', events.timestamp) <= strftime('%s', '2020-01-03 00:00:00')
        AND event_type = 'gleamed the cube'</div>

### Bonus challenge 1

Modify the query above to include not just who gleamed the cube during the given date range, but how many times they did, and sort the results by that number.

In [ ]:
%%sql

/* "Show me users and instance counts for `gleamed the cube` events between January 1st and 3rd (inclusive) */
/* Write your query below, then press Ctrl+Enter to run it! */



<div class="answer">SELECT users.name, count(events.id) as 'cubes gleamed' FROM 
    users JOIN events ON users.id = events.user
    WHERE strftime('%s', events.timestamp) >= strftime('%s', '2020-01-01 00:00:00')
        AND strftime('%s', events.timestamp) <= strftime('%s', '2020-01-03 00:00:00')
        AND event_type = 'gleamed the cube'
    GROUP BY events.user
    ORDER BY 'cubes gleamed'; </div>

### Bonus challenge 2

Rewrite the query again, this time so it has three columns: name, date, and the number of logged events for that user _on that date_, still limited to `gleamed the cube` events between 1 Jan and 3 Jan, e.g.

| name   |	date       |	cubes gleamed |
| ---    | ---         | ---
| shayna |	2020-01-01 |	2 |
| asuka  |	2020-01-01 |	1 |
| bob 	 |  2020-01-02 |	1 |
| jane 	 |  2020-01-02 |	1 |
| shayna |	2020-01-03 |	1 |

This one is intended to be a real challenge, so some hints are in order:
1. What you `GROUP` on here is key
2. You can `GROUP` on multiple columns (comma-separated, order matters!!)
3. Even though our example `timestamp` values all happen to be at midnight, you could have records at multiple times on the same day, so the best solution will use `strftime` ([docs](https://sqlite.org/lang_datefunc.html)) to extract just the YYYY-MM-DD of the timestamp and group on that

In [ ]:
%%sql

/* "Show me users, dates, and per-day instance counts for `gleamed the cube` events between January 1st and 3rd (inclusive) */
/* Write your query below, then press Ctrl+Enter to run it! */



<div class="answer">SELECT users.name, strftime('%Y-%m-%d', events.timestamp) as date, count(events.id) as 'cubes gleamed' FROM 
    users JOIN events ON users.id = events.user
    WHERE strftime('%s', events.timestamp) &gt;= strftime('%s', '2020-01-01 00:00:00')
        AND strftime('%s', events.timestamp) &lt;= strftime('%s', '2020-01-03 00:00:00')
        AND event_type = 'gleamed the cube'
    GROUP BY date, events.user
    ORDER BY date, 'cubes gleamed';</div>

# Optional further reading

For more SQL nuts and bolts, see the w3schools.com [SQL reference](https://www.w3schools.com/sql/default.asp).

For a simple refresher when needed, cheat sheets like [this one](https://www.sqltutorial.org/sql-cheat-sheet/) abound online.
